In [1]:
# regular imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import data 
takehome_user_engagement = pd.read_csv("takehome_user_engagement_2.csv")

In [3]:
# importing as a CSV had issues so I saved the doc as an Excel file and imported that
takehome_users = pd.read_excel("takehome_users_.xls")

### Take a look at the data 

In [42]:
# we see there are 207,917 days of engagement 
takehome_user_engagement.shape

(207917, 2)

In [41]:
takehome_users.shape

(12000, 13)

In [41]:
# we have 12k users total in the takehome_users dataframe
takehome_users.object_id.nunique()

12000

In [4]:
takehome_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:00,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:00,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:00,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:00,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:00,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
takehome_user_engagement.head(2)

,time_stamp,user_id,visited
0,4/22/2014 3:53,1,1
1,11/15/2013 3:45,2,1


In [7]:
# We have 8823 users who engaged at all 
takehome_user_engagement.user_id.nunique()

8823

In [5]:
takehome_user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [6]:
# Turn the time stamp to a date column 
takehome_user_engagement["date"] = pd.to_datetime(takehome_user_engagement["time_stamp"])

In [7]:
takehome_user_engagement.head(4)

,time_stamp,user_id,visited,date
0,4/22/2014 3:53,1,1,2014-04-22 03:53:00
1,11/15/2013 3:45,2,1,2013-11-15 03:45:00
2,11/29/2013 3:45,2,1,2013-11-29 03:45:00
3,12/9/2013 3:45,2,1,2013-12-09 03:45:00


In [10]:
takehome_user_engagement.head()

,time_stamp,user_id,visited,date
0,4/22/2014 3:53,1,1,2014-04-22
1,11/15/2013 3:45,2,1,2013-11-15
2,11/29/2013 3:45,2,1,2013-11-29
3,12/9/2013 3:45,2,1,2013-12-09
4,12/25/2013 3:45,2,1,2013-12-25


#### We only care about the date not the time so we remove that

In [11]:
takehome_user_engagement["date"] = takehome_user_engagement["date"].apply(lambda x: x.date())

In [12]:
# from this we see that the visited column is not needed
takehome_user_engagement.visited.value_counts()

1    207917
Name: visited, dtype: int64

In [13]:
takehome_user_engagement = takehome_user_engagement[["user_id", "date"]]

In [14]:
takehome_user_engagement.head()

,user_id,date
0,1,2014-04-22
1,2,2013-11-15
2,2,2013-11-29
3,2,2013-12-09
4,2,2013-12-25


In [15]:
# we only care about if they visited once in a day so we drop duplicates. 
takehome_user_engagement.drop_duplicates(inplace=True)

In [16]:
takehome_user_engagement.head()

,user_id,date
0,1,2014-04-22
1,2,2013-11-15
2,2,2013-11-29
3,2,2013-12-09
4,2,2013-12-25


In [17]:
# adopted users have to have at least three visits so we remove users with less than three visits. 
v_c= takehome_user_engagement.user_id.value_counts().to_frame()
potential_users= v_c[v_c["user_id"] > 2].index

In [19]:
takehome_user_engagement = takehome_user_engagement[takehome_user_engagement["user_id"].isin(potential_users)]

In [20]:
takehome_user_engagement.user_id.nunique()

2248

### we see from the original 8,823 with user engagement only 2,248 could be adopted users

###  we next create a method to figure out which of these users are adopted. 

In [37]:
# we take one sample user
sample = takehome_user_engagement[takehome_user_engagement["user_id"] == 2]

sample = sample.reset_index()

# by taking the date column and subtracking the previous date from two periods earlier we can see if they 
# had three sessions in a week period. We fill NaN's to 10 days because the first two periods can't tell us anything 
sample["time_diff"]= (sample['date']-sample['date'].shift(periods=2)).fillna('10 days')
sample

,index,user_id,date,time_diff
0,1,2,2013-11-15,10 days
1,2,2,2013-11-29,10 days
2,3,2,2013-12-09,24 days
3,4,2,2013-12-25,26 days
4,5,2,2013-12-31,22 days
5,6,2,2014-01-08,14 days
6,7,2,2014-02-03,34 days
7,8,2,2014-02-08,31 days
8,9,2,2014-02-09,6 days
9,10,2,2014-02-13,5 days


In [40]:
# This will create a list of True or False for the users visits. 
# a true will only appear if they visited at least three times in 7 days, thus indicating that the user is an adopted user. 
t_list= list(sample['time_diff']<"8 days")
t_list

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False]

In [33]:
def find_adopted_users(df):
    # create a list of possible users 
    users = list(df.user_id.unique())
    # create an empty list for the adopted users 
    adopted_users_list=[]
    
    # run through each potential user in the dataframe
    for user in users:
        sample = df[df["user_id"] == user].reset_index()
        # check for time between every three visits
        sample["time_diff"]= (sample['date']-sample['date'].shift(periods=2)).fillna('10 days')
        # check if the time is within a week
        t_list= list(sample['time_diff']<"8 days")
        # if they had three visits in a week, add them to the adopted_users_list 
        if True in t_list:
            adopted_users_list.append(user)
    return adopted_users_list

In [34]:
adopted_users = find_adopted_users(takehome_user_engagement)

In [36]:
len(adopted_users)

1656

In [ ]:
# we see that only 1,656 users are adopted users. 

In [28]:
takehome_users.head(1)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:00,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0


In [43]:
takehome_users["adopted_users"] = takehome_users["object_id"].isin(adopted_users) 

In [82]:
takehome_users["adopted_num"] = takehome_users["adopted_users"] == True 

In [83]:
takehome_users.head(1)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_users,creation_year,creation_month,adopted_num
0,1,2014-04-22 03:53:00,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False,2014,4,False


In [46]:
takehome_users["creation_year"] = takehome_users["creation_time"].apply(lambda x:x.year)

In [47]:
takehome_users["creation_month"] = takehome_users["creation_time"].apply(lambda x:x.month)

In [48]:
takehome_users.adopted_users.value_counts()

False    10344
True      1656
Name: adopted_users, dtype: int64

In [54]:
# 13.8% of all users are adopted users 
1656/12000*100

13.8

In [55]:
# 18.8% of all users who logged on are adopted users  
1656/8823*100

18.769126147568855

In [56]:
# 73.7% of all users who had three or more vists are adopted users 
1656/2248*100

73.66548042704626

### Compare the adopted users to the non-adopted users

#### examine percent of takehome_users who opted in to the mailling list

In [67]:
takehome_users.groupby("adopted_users")[["opted_in_to_mailing_list"]].mean()* 100

,opted_in_to_mailing_list
adopted_users,
False,24.796984
True,25.905797


#### examine percent of takehome_users who enabled for marketing drip

In [68]:
takehome_users.groupby("adopted_users")[["enabled_for_marketing_drip"]].mean()* 100

,enabled_for_marketing_drip
adopted_users,
False,14.839520
True,15.519324


#### Examine adoption rate by creation year

In [128]:
takehome_users.groupby("creation_year")[["adopted_users"]].mean()* 100

,adopted_users
creation_year,
2012,16.732213
2013,15.415786
2014,8.874398


## Creation Source Evaluated

In [115]:
takehome_users.groupby("creation_source")[["adopted_users"]].mean()* 100

,adopted_users
creation_source,
GUEST_INVITE,17.059639
ORG_INVITE,13.493183
PERSONAL_PROJECTS,8.147797
SIGNUP,14.470532
SIGNUP_GOOGLE_AUTH,17.256318


## examine creation month's affect on a users adoption

In [130]:
cray_mo = takehome_users.groupby("creation_month")[["adopted_users"]].mean()* 100
cray_mo.reset_index().sort_values('adopted_users', ascending=False )

,creation_month,adopted_users
5,6,18.192628
9,10,17.532468
7,8,17.018779
10,11,16.132479
8,9,15.760266
1,2,14.668094
11,12,14.623656
0,1,14.577259
6,7,14.436620
2,3,13.933730


## Examine org_id  

In [125]:
org_id_df = takehome_users.groupby("org_id")[["adopted_users"]].mean()* 100
org_id_df.reset_index().sort_values('adopted_users', ascending=False )

,org_id,adopted_users
387,387,58.333333
235,235,46.153846
270,270,42.857143
399,399,38.461538
400,400,37.500000
392,392,37.500000
415,415,37.500000
117,117,36.363636
345,345,35.714286
291,291,35.294118


## invited_by_user_id  

In [135]:
takehome_users.invited_by_user_id.value_counts().to_frame().reset_index().invited_by_user_id.value_counts()

1     1097
2      527
3      347
4      205
5      165
6       98
7       60
8       36
9       14
10      10
11       3
13       1
12       1
Name: invited_by_user_id, dtype: int64

In [126]:
org_id_df = takehome_users.groupby("invited_by_user_id")[["adopted_users"]].mean()* 100
org_id_df.reset_index().sort_values('adopted_users', ascending=False )

,invited_by_user_id,adopted_users
1080,5022.0,100.0
180,907.0,100.0
1292,5925.0,100.0
460,2188.0,100.0
1287,5910.0,100.0
923,4276.0,100.0
2234,10388.0,100.0
1279,5875.0,100.0
2236,10405.0,100.0
1274,5855.0,100.0
